In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

def ideal_gmf_correlation(wavelengths, incidence_angle=36.0):
    """
    Simulate correlation between B0 and wind PSD for a perfect scale-invariant GMF.
    
    For a perfect GMF:
    - B0 responds to mean wind speed squared (total energy)
    - PSD at each k represents energy at that scale
    - Correlation depends on how much each scale contributes to mean wind
    """
    
    # Convert wavelength to wavenumber
    k = 2 * np.pi / wavelengths
    k_normalized = k / k[0]  # Normalize by lowest k
    
    # For a perfect GMF, B0 measures mean wind energy
    # Mean wind is dominated by large scales (low k)
    
    # 1. Correlation with mean wind component (large scales)
    # Uses Kolmogorov -5/3 spectrum weighting
    mean_wind_contribution = 1 / (1 + (k_normalized)**1.67)
    
    # 2. For a perfect GMF, there's no artificial scale separation
    # B0 should see integrated effect of all scales weighted by their energy
    
    # Theoretical wind spectrum follows k^(-5/3) in inertial range
    spectral_weight = k_normalized**(-5/3)
    spectral_weight[0] = spectral_weight[1]  # Avoid infinity at k=0
    
    # Normalize weights
    spectral_weight = spectral_weight / np.sum(spectral_weight)
    
    # B0 correlation with PSD at each scale
    # High at low k (large scales dominate mean wind)
    # Decreases smoothly at high k (small scales contribute less to mean)
    corr_B0 = mean_wind_contribution * 0.95 + 0.05
    
    # B1 and B2 in perfect GMF would show similar patterns but with opposite sign
    # They modulate based on directional asymmetry
    corr_B1 = -0.3 * (1 - mean_wind_contribution) * np.cos(np.radians(incidence_angle)/36)
    corr_B2 = -0.6 * (1 - mean_wind_contribution) * np.cos(2*np.radians(incidence_angle)/36)
    
    return corr_B0, corr_B1, corr_B2

def plot_ideal_vs_actual(wavelengths, actual_corr_B0, actual_corr_B1, actual_corr_B2):
    """
    Plot ideal correlations vs actual observed correlations
    """
    ideal_B0, ideal_B1, ideal_B2 = ideal_gmf_correlation(wavelengths)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
    
    # Plot ideal case
    ax1.semilogx(wavelengths, ideal_B0, 'r-', linewidth=2, label='B0 (ideal)')
    ax1.semilogx(wavelengths, ideal_B1, 'g-', linewidth=2, label='B1 (ideal)')
    ax1.semilogx(wavelengths, ideal_B2, 'b-', linewidth=2, label='B2 (ideal)')
    ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax1.set_xlabel('Wavelength [m]')
    ax1.set_ylabel('Correlation')
    ax1.set_title('Ideal Scale-Invariant GMF Correlations')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(-1, 1)
    
    # Plot comparison
    ax2.semilogx(wavelengths, ideal_B0, 'r-', linewidth=2, alpha=0.5, label='B0 (ideal)')
    ax2.semilogx(wavelengths, actual_corr_B0, 'r--', linewidth=2, label='B0 (actual)')
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    
    # Highlight the problematic region
    ax2.axvspan(500, 2000, alpha=0.2, color='yellow', label='Scale transition zone')
    
    ax2.set_xlabel('Wavelength [m]')
    ax2.set_ylabel('Correlation')
    ax2.set_title('B0 Correlation: Ideal vs Actual')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(-0.2, 1.1)
    
    plt.tight_layout()
    return fig

# Example usage with synthetic "actual" data mimicking your observation
wavelengths = np.linspace(100, 22000, 200)

# Simulate your observed pattern
k = 2 * np.pi / wavelengths
actual_B0 = 0.85 - 0.4 * np.exp(-((wavelengths - 1000)/300)**2) + 0.1 * wavelengths/wavelengths.max()
actual_B1 = -0.1 - 0.5 * (1 - np.exp(-wavelengths/5000))
actual_B2 = -0.6 - 0.2 * (1 - np.exp(-wavelengths/5000))

# Plot comparison
fig = plot_ideal_vs_actual(wavelengths, actual_B0, actual_B1, actual_B2)
plt.show()

# Additional analysis function
def explain_correlation_physics(wavelengths):
    """
    Explain what each correlation means physically
    """
    ideal_B0, ideal_B1, ideal_B2 = ideal_gmf_correlation(wavelengths)
    
    print("Physical Interpretation of Ideal Correlations:")
    print("=" * 50)
    print(f"1. B0 at λ=100m: {ideal_B0[0]:.3f}")
    print("   → Near perfect correlation because small scales integrate to mean wind")
    print(f"2. B0 at λ=10km: {ideal_B0[-50]:.3f}")  
    print("   → Still high because large scales directly represent mean wind variations")
    print(f"3. B1/B2: Negative because they represent directional modulations")
    print("   → Opposition to isotropic PSD energy")
    
    # Where the actual deviates from ideal
    k_transition = 2 * np.pi / 1000  # 1km wavelength
    print(f"\nIn actual CMOD5N:")
    print(f"- Sharp drop at λ≈1km indicates scale-dependent GMF breakdown")
    print(f"- The GMF cannot properly handle the transition between:")
    print(f"  • Sea surface roughness (λ < 1km)")
    print(f"  • Mean wind modulation (λ > 1km)")

explain_correlation_physics(wavelengths)